In [1]:
import zipfile
import pandas as pd
import numpy as np
from PIL import Image
import io
import cv2  # Optional (if you prefer OpenCV)
import json
import os
from sklearn.model_selection import train_test_split #train-test split
import shutil

c:\Users\Mert\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


#### **MultiOFF**

In [2]:
dataframes = {}  # Stores CSV data with filenames as keys

zip_path = "../external_data/MultiOFF/annotations.zip"

try:
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        # List all CSV files in the ZIP archive
        csv_files = [f for f in zip_ref.namelist() if f.endswith('.csv')]
        
        if not csv_files:
            print("No CSV files found in the ZIP archive.")
        else:
            for csv_file in csv_files:
                with zip_ref.open(csv_file) as f:
                    # Read CSV into a DataFrame
                    df = pd.read_csv(f)
                    dataframes[csv_file] = df
                    print(f"Successfully read: {csv_file}")

except FileNotFoundError:
    print(f"Error: The file {zip_path} does not exist.")
except zipfile.BadZipFile:
    print("Error: The ZIP file is corrupted or invalid.")

Successfully read: Training_meme_dataset.csv
Successfully read: Testing_meme_dataset.csv
Successfully read: Validation_meme_dataset.csv


In [3]:
training_multioff = dataframes["Training_meme_dataset.csv"]
validation_multioff = dataframes["Validation_meme_dataset.csv"]
test_multioff = dataframes["Testing_meme_dataset.csv"]

#Remove the label column for each dataset
training_multioff = training_multioff.drop(columns=["label"])
validation_multioff = validation_multioff.drop(columns=["label"])
test_multioff = test_multioff.drop(columns=["label"])

#Rename the columns for each dataset: {image_name:image,sentence:text}
training_multioff = training_multioff.rename(columns={"image_name": "image", "sentence": "text"})
validation_multioff = validation_multioff.rename(columns={"image_name": "image", "sentence": "text"})
test_multioff = test_multioff.rename(columns={"image_name": "image", "sentence": "text"})


print(training_multioff.head())
print(validation_multioff.head())
print(test_multioff.head())


#Concatenate all the dataframes into one
all_data = pd.concat([training_multioff, validation_multioff, test_multioff], ignore_index=True)
# Reset the index of the concatenated dataframe
all_data.reset_index(drop=True, inplace=True)

#Toal length of the datasets combined together written explicity
print("\n\n Total length of the datasets combined together:",len(all_data))


             image                                               text
0  LJ3r8Gy.jpg.png  OFFICIAL BERNIE SANDERS DRINKING GAME ! Every ...
1      qDnIIHA.png  2:28 PM THIS IS A WALL INSIDE A NAZI GAS CHAMB...
2      1JQk5NF.png                o shit waddup ! BERNIE SANDERS COM 
3        iMMNq.png  `` MITT ROMNEY IS THE WORST REPUBLICAN IN THE ...
4      jAi3iI1.png  Anonymous ( ID : duqdA1io a 08/05/16 ( Fri ) 1...
         image                                               text
0  Xxc4mjq.png  WE LIKE IKE I LIKE IKE FRANK CULOTTA REPUBLICA...
1  ZffTHk2.png                                   Glory to Bern . 
2  TyYDiSx.png  My mom got kicked out of her emotionally abusi...
3  FsAbNtn.png          J. TRUMP DONALD MA DE N MEXIC i RN 47333 
4  ERe2yR0.png  score hidden ] 5 hours ago My friend committed...
         image                                               text
0  jyxHhiB.png  3 hrs Black nurse in Connecticut asked me if T...
1  we4hhWi.png  I do n't believe that women have any

In [4]:
zip_path = '../external_data/MultiOFF/labelled_images.zip'
subfolder = 'Labelled Images/'  # Must match the exact casing in the ZIP

# Simple list to store PIL images
image_list_mff = []

with zipfile.ZipFile(zip_path, 'r') as zip_file:
    # Get all files in the subfolder with common image extensions
    for file in zip_file.namelist():
        if file.startswith(subfolder) and file.lower().endswith(('.png', '.jpg', '.jpeg')):
            with zip_file.open(file) as img_file:
                try:
                    img = Image.open(io.BytesIO(img_file.read())).convert('RGB')
                    image_list_mff.append(img)
                    print(f"Loaded: {file.split('/')[-1]}")  # Show filename without path
                except Exception as e:
                    print(f"Skipped {file}: {str(e)}")

print(f"\nTotal images loaded: {len(image_list_mff)}")


Loaded: z5sSuk3.png
Loaded: zQPb3nD.png
Loaded: ykGRBWn.png
Loaded: ZMaAKc9.png
Loaded: Zh4CZSe.png
Loaded: zOImoh5.png
Loaded: ZOSUjXw.png
Loaded: zRgnrbT.png
Loaded: z1gozo1.png
Loaded: ZEjNdo0.png
Loaded: yZLLqn4.png
Loaded: zKbUmLB.png
Loaded: yWd3NEB.png
Loaded: ZbqRtuL.png
Loaded: ZwFfyVJ.png
Loaded: zq7bXGu.png
Loaded: ysAk127.png
Loaded: zoIImWJ.png
Loaded: ZWajHQq.png
Loaded: zZIOwqg.png
Loaded: YJ1QbDX.png
Loaded: zFnJ1x3.png
Loaded: zPT9dFf.png
Loaded: zr5VeQM.png
Loaded: yLF1GSc.png
Loaded: zl617iP.png
Loaded: YW7mx62.png
Loaded: Z1VeVXi.png
Loaded: zAUFD7y.png
Loaded: zs6maXI.png
Loaded: y898T2X.png
Loaded: ztdf8dZ.png
Loaded: ZAloCy9.png
Loaded: xGBX7zN.png
Loaded: XMReWCt.png
Loaded: ykaOeQZ.png
Loaded: xYTrJJp.png
Loaded: xtvEjIx.png
Loaded: ZffTHk2.png
Loaded: xrh5RQD.png
Loaded: yiS7xVy.png
Loaded: yGHKtzg.png
Loaded: xLXvr9t.png
Loaded: YIi9Bga.png
Loaded: XYERAZg.png
Loaded: yDO396v.png
Loaded: XwccSUy.png
Loaded: xJiXDnY.png
Loaded: xyi7yMK.png
Loaded: xQkfJse.png


#### **HarMeme**

In [5]:

annotations_dir = '../external_data/HarMeme/mmf/data/datasets/memes/defaults/annotations'

har_data = {}  # Stores data with filenames as keys

# Check if the directory exists
if not os.path.isdir(annotations_dir):
    print(f"Directory not found: {annotations_dir}")
else:
    # List all .jsonl files in the directory
    jsonl_files = [f for f in os.listdir(annotations_dir) if f.endswith('.jsonl')]
    
    if not jsonl_files:
        print(f"No .jsonl files found in {annotations_dir}")
    else:
        for file_name in jsonl_files:
            file_path = os.path.join(annotations_dir, file_name)
            har_data[file_name] = []
            
            # Read each line of the JSONL file
            with open(file_path, 'r', encoding='utf-8') as f:
                for line in f:
                    try:
                        # Parse each line as a JSON object
                        json_obj = json.loads(line)
                        har_data[file_name].append(json_obj)
                    except json.JSONDecodeError as e:
                        print(f"Error parsing line in {file_name}: {str(e)}")
            
            print(f"Loaded {len(har_data[file_name])} entries from {file_name}")

train_har = pd.DataFrame(har_data["train.jsonl"])
val_har = pd.DataFrame(har_data["val.jsonl"])
test_har = pd.DataFrame(har_data["test.jsonl"])

#Concatenate all the dataframes into one
all_har_data = pd.concat([train_har, val_har, test_har], ignore_index=True)

# Drop columns other than image_id and sentence
all_har_data = all_har_data[['image', 'text']]

print("\n\n Total length of the datasets combined together:",len(all_har_data))

Loaded 354 entries from test.jsonl
Loaded 3013 entries from train.jsonl
Loaded 177 entries from val.jsonl


 Total length of the datasets combined together: 3544


In [6]:
image_dir = '../external_data/HarMeme/mmf/data/datasets/memes/defaults/images'

images_har = []
valid_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.gif', '.tiff')

for filename in os.listdir(image_dir):
    if filename.lower().endswith(valid_extensions):
        img_path = os.path.join(image_dir, filename)
        try:
            img = Image.open(img_path)  # RGB format
            images_har.append(img)
        except Exception as e:
            print(f"Error loading {filename}: {str(e)}")

print(f"Loaded {len(images_har)} images.")

Loaded 3544 images.


#### **Fine-Tuning the TrOCR with HarMeme and Multioff datasets combined**

In [7]:
# View the data in Multioff
print(all_data.head())
print("\n\n",image_list_mff[:5])

             image                                               text
0  LJ3r8Gy.jpg.png  OFFICIAL BERNIE SANDERS DRINKING GAME ! Every ...
1      qDnIIHA.png  2:28 PM THIS IS A WALL INSIDE A NAZI GAS CHAMB...
2      1JQk5NF.png                o shit waddup ! BERNIE SANDERS COM 
3        iMMNq.png  `` MITT ROMNEY IS THE WORST REPUBLICAN IN THE ...
4      jAi3iI1.png  Anonymous ( ID : duqdA1io a 08/05/16 ( Fri ) 1...


 [<PIL.Image.Image image mode=RGB size=617x529 at 0x245FAC25790>, <PIL.Image.Image image mode=RGB size=2048x1653 at 0x245B9E73C50>, <PIL.Image.Image image mode=RGB size=500x493 at 0x245B9FF3190>, <PIL.Image.Image image mode=RGB size=822x551 at 0x245FAE4FA10>, <PIL.Image.Image image mode=RGB size=960x809 at 0x245FAF46790>]


In [8]:
# View the data in HarMeme
print(all_har_data.head())
print("\n\n",images_har[:5])

                 image                                               text
0   covid_memes_18.png  Bernie or Elizabeth?\nBe informed.Compare them...
1   covid_memes_19.png  Extending the\nBrexit deadline until\nOctober ...
2  covid_memes_252.png  kwai\ngkwa 0964\n#nnevvy\napplause to Thais fr...
3  covid_memes_255.png  So, I order this\nfoce mask to\nprotect ogains...
4   covid_memes_20.png  best candidate for\nJA\n2020\njoe biden\nKamal...


 [<PIL.PngImagePlugin.PngImageFile image mode=RGB size=640x643 at 0x245FAE4CE10>, <PIL.PngImagePlugin.PngImageFile image mode=RGB size=235x215 at 0x245FAE4F110>, <PIL.PngImagePlugin.PngImageFile image mode=RGB size=226x223 at 0x245FAE4CA50>, <PIL.PngImagePlugin.PngImageFile image mode=RGB size=268x188 at 0x245FAE2B590>, <PIL.PngImagePlugin.PngImageFile image mode=RGB size=231x218 at 0x245FAC13990>]


In [9]:
def split_train_test(all_data, test_size=0.2, random_state=42):
    # Split the data
    train_df, test_df = train_test_split(
        all_data,
        test_size=test_size,
        random_state=random_state
    )
    
    # Reset indices
    train_df = train_df.reset_index(drop=True)
    test_df = test_df.reset_index(drop=True)
    
    # Print dataset sizes
    print("Train set size:", len(train_df))
    print("Test set size:", len(test_df))
    
    return train_df, test_df

In [10]:
train_df_mff, test_df_mff = split_train_test(all_data)
train_df_har, test_df_har = split_train_test(all_har_data)


#Concatenate the training data
train_df = pd.concat([train_df_mff, train_df_har], ignore_index=True)
#Concatenate the test data
test_df = pd.concat([test_df_mff, test_df_har], ignore_index=True)
# Reset the index of the concatenated dataframe
train_df.reset_index(drop=True, inplace=True)
test_df.reset_index(drop=True, inplace=True)


# Print the final dataset sizes
print("Final Train set size:", len(train_df))
print("Final Test set size:", len(test_df))

Train set size: 594
Test set size: 149
Train set size: 2835
Test set size: 709
Final Train set size: 3429
Final Test set size: 858


In [11]:
print(train_df.head())
print(test_df.head())

         image                                               text
0  X2dmDrp.png  `` Chelsea Clinton steps out of college and he...
1  rOvVkF1.png  o 28 % 1:19 PM 76 AT & T M-Cell Details Dad Me...
2  m4oY5vK.png  On 1-20-2017 What is the first thing you want ...
3  Okb6dJB.png  N Dallas , Texas 10:31 AM CT DALLAS TRUMP VS ....
4  4KXGjz0.png  `` One thing ca n't stand is the military . It...
         image                                               text
0  12XLnzK.png  thewrap.com SUICIDE `` Suicide Squad ' Executi...
1  VAJZJJV.png  3 hrs This was in my Facebook feed this mornin...
2  zvdzU3y.png  Bernie or Hillary ? Be informed . Compare them...
3  qi0bYgG.png  WE 'RE FUCKED , MOVE TO CANADA I STILL FEEL YO...
4  NpsCjMn.png  SAYS BERNIE SANDERS SUPPORTERS ARE LAZY AND UN...


In [12]:
#Create a folder called MMF_HAR_comb_processed in data folder
#Create a folder ../data/processed/MMF_HAR_comb_processed/training
# and ../data/processed/MMF_HAR_comb_processed/testing
if not os.path.exists('../data/processed/MMF_HAR_comb_processed'):
    os.makedirs('../data/processed/MMF_HAR_comb_processed')
if not os.path.exists('../data/processed/MMF_HAR_comb_processed/training'):
    os.makedirs('../data/processed/MMF_HAR_comb_processed/training')
if not os.path.exists('../data/processed/MMF_HAR_comb_processed/testing'):
    os.makedirs('../data/processed/MMF_HAR_comb_processed/testing')

# Save the combined train and test sets to CSV files
train_df.to_csv('../data/processed/MMF_HAR_comb_processed/training/mmf_har_train_combined.csv', index=False)
test_df.to_csv('../data/processed/MMF_HAR_comb_processed/testing/mmf_har_test_combined.csv', index=False)

In [36]:
print('################################################################')
#go throught the image column in test_df and check is the image is 
print("Total length of the dataframe: ",len(all_data) + len(all_har_data))
#Check the length of the images in the folder: 'data/processed/MMF_HAR_comb_processed/images/general' 
print("Number of total images: ", len(os.listdir('../data/processed/MMF_HAR_comb_processed/images/general')))
print('################################################################')


#Concatenate all_df and all_har_data
all_df = pd.concat([all_data, all_har_data], ignore_index=True)

#Check if all the image names in folder above are in the dataframe "image" column
#Get the image names that are not in the all_df image column
not_in_df = [img for img in os.listdir('../data/processed/MMF_HAR_comb_processed/images/general') if img not in all_df['image'].values]
print("Images not in DataFrame:", not_in_df)
print('################################################################')


#Remove the images that are not in the dataframe from the folder 'data/processed/MMF_HAR_comb_processed/images/general'
for img in not_in_df:
    os.remove(os.path.join('../data/processed/MMF_HAR_comb_processed/images/general', img))

#Check if the images are removed from the folder
print("Images removed from folder: ", len(os.listdir('../data/processed/MMF_HAR_comb_processed/images/general')))
#Check if the images are in the dataframe
print("Images in DataFrame: ", len(all_df['image'].values))
print('################################################################')


#Write the code that seperates the images in the train_df and test_df and store them in lists
train_images = []
test_images = []
for img in train_df['image'].values:
    train_images.append(img)
for img in test_df['image'].values:
    test_images.append(img)

print("Train images: ", len(train_images))
print("Test images: ", len(test_images))
print('################################################################')

# Now create a folder called test in 'data/processed/MMF_HAR_comb_processed/images/general 
# Training in 'data/processed/MMF_HAR_comb_processed/images/general and store those images in those folders

if not os.path.exists('../data/processed/MMF_HAR_comb_processed/images/training'):
    os.makedirs('../data/processed/MMF_HAR_comb_processed/images/training')

if not os.path.exists('../data/processed/MMF_HAR_comb_processed/images/testing'):
    os.makedirs('../data/processed/MMF_HAR_comb_processed/images/testing')

# Copy training images
for img in train_images:
    src = os.path.join('../data/processed/MMF_HAR_comb_processed/images/general', img)
    dst = os.path.join('../data/processed/MMF_HAR_comb_processed/images/training', img)
    shutil.copy(src, dst)
# Copy testing images
for img in test_images:
    src = os.path.join('../data/processed/MMF_HAR_comb_processed/images/general', img)
    dst = os.path.join('../data/processed/MMF_HAR_comb_processed/images/testing', img)
    shutil.copy(src, dst)


#Check if the images are copied to the folders
print("Training images copied to: ", len(os.listdir('../data/processed/MMF_HAR_comb_processed/images/training')))
print("Testing images copied to: ", len(os.listdir('../data/processed/MMF_HAR_comb_processed/images/testing')))
print('################################################################')




################################################################
Total length of the dataframe:  4287
Number of total images:  4287
################################################################
Images not in DataFrame: []
################################################################
Images removed from folder:  4287
Images in DataFrame:  4287
################################################################
Train images:  3429
Test images:  858
################################################################
Training images copied to:  3429
Testing images copied to:  858
################################################################
